# Data Preparation (CRISP-DM)

Nesta etapa do **CRISP-DM**, os dados brutos são transformados em uma base estruturada, consistente e adequada para análise. O processo envolve **limpeza**, **padronização**, **tratamento de valores ausentes**, **conversão de tipos**, **engenharia de variáveis** e **anonimização** de informações sensíveis.

O objetivo é construir um dataset **confiável, reprodutível e pronto para uso**, servindo como insumo direto para as etapas de **Exploratory Data Analysis (EDA)** e eventuais fases posteriores de modelagem ou visualização.

In [2]:
import re
import hashlib
import unicodedata
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.shape

(24009, 16)

In [4]:
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^a-z0-9_]", "_", regex=True)
    )

    return df

df.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Renomeando colunas

In [5]:
# Criando df_prepared como cópia de df_raw
df_prepared = df.copy()

# renomeando colunas
df_prepared.rename(columns={
    "Referência": "referencia",
    "Nome": "nome",
    "Cargo": "cargo",
    "Data Admissão": "data_admissao",
    "Tipo de Regime": "tipo_regime",
    "Descontos": "descontos",
    "Liquido": "liquido",
    "Data Desligamento": "data_desligamento",
    "Proventos": "proventos",
    "Contrato": "contrato",
    "Atividade": "atividade",
    "Nome Atividade": "nome_atividade",
    "Tipo de Contrato": "tipo_contrato",
    "Data Prevista Termino Contrato": "data_prevista_termino_contrato",
    "Carga Horária (Sem.)": "carga_horaria_semanal"
}, inplace=True)

In [6]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


In [7]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24008,NaN,NaN,NaN,NaN,NaN,"918.286,26","4.302.011,08",NaN,"5.220.297,34",NaN,NaN,NaN,NaN,NaN,NaN,set2025.csv


## Removendo as linhas de somas finais que vem em cada dataset

In [8]:
df_prepared = df_prepared.dropna(subset=["referencia"])

In [9]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
24003,Folha Mensal - Setembro,YASMIN MICKAELA TAMINI,AGENTE DE DESENVOLVIMENTO INFANTIL,17/01/2025,REGIME PROPRIO,"535,56","2.920,63",NaN,"3.456,19",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv


In [10]:
# Extrair tipo de pagamento e mês diretamente da coluna referencia
df_prepared[["tipo_pagamento_raw", "mes_referencia"]] = df_prepared["referencia"].str.split(" - ", expand=True)

# Padronizar tipo de pagamento
df_prepared["tipo_pagamento"] = df_prepared["tipo_pagamento_raw"].replace({
    "Folha Mensal": "folha_mensal",
    "Folha Complementar": "vale_alimentacao"
})

# remover coluna temporária
df_prepared = df_prepared.drop(columns=["tipo_pagamento_raw"])

# conferir resultados
df_prepared[["referencia", "tipo_pagamento", "mes_referencia"]].head()

,referencia,tipo_pagamento,mes_referencia
0,Folha Complementar - Abril,vale_alimentacao,Abril
1,Folha Mensal - Abril,folha_mensal,Abril
2,Folha Complementar - Abril,vale_alimentacao,Abril
3,Folha Mensal - Abril,folha_mensal,Abril
4,Folha Complementar - Abril,vale_alimentacao,Abril


In [11]:
df_prepared["tipo_pagamento"].unique()

array(['vale_alimentacao', 'folha_mensal', 'Adiantamento 13º Salário',
       'Folha Complementar c/ Encargos', 'Rescisão',
       'Fechamento 13º Salário'], dtype=object)

In [12]:
def normalizar_tipo_pagamento(valor):
    if pd.isna(valor):
        return valor
    
    mapeamento = {
        "vale_alimentacao": "vale_alimentacao",
        "folha_mensal": "folha_mensal",
        "Adiantamento 13º Salário": "adiantamento_13_salario",
        "Folha Complementar c/ Encargos": "folha_complementar_com_encargos",
        "Rescisão": "rescisao",
        "Fechamento 13º Salário": "fechamento_13_salario"
    }

    return mapeamento.get(valor, valor)

In [13]:
df_prepared["tipo_pagamento"].apply(normalizar_tipo_pagamento)

0        vale_alimentacao
1            folha_mensal
2        vale_alimentacao
3            folha_mensal
4        vale_alimentacao
               ...       
24003        folha_mensal
24004    vale_alimentacao
24005        folha_mensal
24006        folha_mensal
24007    vale_alimentacao
Name: tipo_pagamento, Length: 23997, dtype: object

In [14]:
df_prepared["tipo_pagamento"].unique()

array(['vale_alimentacao', 'folha_mensal', 'Adiantamento 13º Salário',
       'Folha Complementar c/ Encargos', 'Rescisão',
       'Fechamento 13º Salário'], dtype=object)

In [15]:
df_prepared["tipo_pagamento_normalizado"] = df_prepared["tipo_pagamento"].apply(normalizar_tipo_pagamento)

In [16]:
df_prepared["tipo_pagamento_normalizado"].value_counts()

tipo_pagamento_normalizado
folha_mensal                       11005
vale_alimentacao                   10937
adiantamento_13_salario              926
fechamento_13_salario                872
rescisao                             155
folha_complementar_com_encargos      102
Name: count, dtype: int64

## Inferência Manual do Gênero dos Servidores — Processo

Algumas análises do projeto exigem a variável **gênero**, que não está presente nos dados originais. Por esse motivo, foi adotado um processo controlado de **inferência manual**, baseado exclusivamente nos **nomes dos servidores**. A seguir, descreve-se o procedimento completo para garantir transparência, rastreabilidade e reprodutibilidade.

### 1. Extração e deduplicação dos nomes
- Os nomes dos servidores foram extraídos do dataset consolidado.
- Foi aplicada deduplicação para garantir que cada servidor fosse analisado apenas uma vez.
- O objetivo é evitar divergências e reduzir o esforço manual.

### 2. Normalização dos nomes
Para padronizar e facilitar comparações:
- Conversão para letras maiúsculas.
- Remoção de espaços em branco indesejados.
- Essa normalização garante consistência tanto na exportação quanto no *merge* final.

### 3. Criação de DataFrame auxiliar
Foi criado um DataFrame exclusivo contendo:
- **Nome do servidor (normalizado)**
- **Campo destinado à inferência manual (`genero_inferido`)**

Esse DataFrame tornou-se a **fonte oficial** da variável inferida.

### 4. Exportação para CSV
- O DataFrame auxiliar foi exportado para CSV.
- A intenção foi permitir uma análise manual mais eficiente por meio de ferramentas como Excel ou LibreOffice.
- Isso garante rapidez na rotulagem e facilidade de revisão.

### 5. Inferência manual do gênero
- Cada nome foi analisado individualmente.
- A inferência foi feita com base no **primeiro nome**, seguindo convenção simples:
  - `M` para masculino  
  - `F` para feminino
- A inferência manual foi adotada pela maior precisão em relação a métodos automatizados baseados em listas de nomes.

### 6. Reimportação do arquivo tratado
- Após a rotulagem, o CSV atualizado foi reimportado.
- A normalização dos nomes foi reaplicada para assegurar correspondência exata com o dataset original.

### 7. Integração com o dataset principal
- O atributo `sexo_inferido` foi incorporado por meio de **merge**, utilizando o nome normalizado como chave.
- A junção foi validada para garantir ausência de duplicidades ou perdas de informação.

### 8. Validação final
- Verificou-se a existência de valores nulos após o *merge*.
- Caso encontrados, os registros foram revisados diretamente no CSV de apoio.
- O processo foi repetido até que todos os servidores possuíssem classificação consistente.

---

Este procedimento garante **clareza, transparência e reprodutibilidade**, permitindo utilizar a variável `genero_inferido` nas análises subsequentes com segurança metodológica.

In [17]:
nomes_unicos = (
    df_prepared["nome"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

nomes_unicos

0      ACACIO DONIZETTI DE OLIVEIRA
1                 ADA ESTER ARCHILA
2           ADAO RODRIGUES DE JESUS
3                 ADELINI PELINGRIN
4                ADEMILTON MONTEIRO
                   ...             
975            WILSON RANOEL VERGNA
976               YARA MARIA ISABEL
977          YASMIN MICKAELA TAMINI
978           ZAIRA ANGELINA ROGADO
979                   ZILDA MESSINA
Name: nome, Length: 980, dtype: object

In [18]:
df_sexo_servidores = pd.DataFrame({
    "nome_servidor": nomes_unicos,
    "genero_inferido": pd.NA
})

df_sexo_servidores

,nome_servidor,genero_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,<NA>
1,ADA ESTER ARCHILA,<NA>
2,ADAO RODRIGUES DE JESUS,<NA>
3,ADELINI PELINGRIN,<NA>
4,ADEMILTON MONTEIRO,<NA>
...,...,...
975,WILSON RANOEL VERGNA,<NA>
976,YARA MARIA ISABEL,<NA>
977,YASMIN MICKAELA TAMINI,<NA>
978,ZAIRA ANGELINA ROGADO,<NA>


In [19]:
path_output = Path("../data/interim/sexo_servidores.csv")

df_sexo_servidores.to_csv(
    path_output,
    index=False,
    encoding="utf-8"
)

In [20]:
df_sexo_serv = pd.read_csv("../data/interim/inf_sexo_servidor.csv", sep=";")

df_sexo_serv.head()

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ADA ESTER ARCHILA,F
2,ADAO RODRIGUES DE JESUS,M
3,ADELINI PELINGRIN,F
4,ADEMILTON MONTEIRO,M


In [21]:
def normalizar_nome(nome):
    return (
        nome
        .str.upper()
        .str.strip()
    )

df_prepared["nome_servidor_norm"] = normalizar_nome(df_prepared["nome"])
df_sexo_serv["nome_servidor_norm"] = normalizar_nome(df_sexo_serv["nome_servidor"])

In [22]:
df_prepared = df_prepared.merge(
    df_sexo_serv[["nome_servidor_norm", "sexo_inferido"]],
    on="nome_servidor_norm",
    how="left"
)

In [23]:
df_prepared["sexo_inferido"].value_counts(dropna=False)

sexo_inferido
F    14784
M     9213
Name: count, dtype: int64

In [24]:
df_prepared[["nome", "sexo_inferido"]].head()

,nome,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ACACIO DONIZETTI DE OLIVEIRA,M
2,ADA ESTER ARCHILA,F
3,ADA ESTER ARCHILA,F
4,ADAO RODRIGUES DE JESUS,M


## Anonimização dos Dados dos Servidores

A partir desta etapa, os nomes dos servidores deixam de ser utilizados diretamente no processo analítico. Embora os dados sejam públicos, a anonimização é adotada como **boa prática em Ciência de Dados**, garantindo responsabilidade no uso da informação e alinhamento às diretrizes da **LGPD**.

Essa estratégia reforça que o objetivo do projeto é analisar **padrões, distribuições e comportamentos agregados**, evitando qualquer exposição desnecessária de informações pessoais e tornando o fluxo reutilizável em outros contextos e bases sensíveis.

---

### Estratégia Adotada

A anonimização foi realizada por meio da geração de um **identificador único irreversível** para cada servidor, utilizando uma função de hash criptográfico.  
Principais características da abordagem:

- O mesmo nome sempre gera o mesmo identificador (processo determinístico).
- Não é possível reverter o hash para recuperar o nome original.
- O identificador pode ser utilizado com segurança como **chave de junção** ou identificação durante todo o projeto.
- A anonimização não compromete as análises, pois mantém integridade relacional.

O identificador criado passa a ser denominado **`id_servidor`**.

---

### Algoritmo Utilizado

Para geração dos identificadores, foi utilizado o algoritmo **SHA-256**, disponível na biblioteca padrão do Python (`hashlib`).  
Motivos para a escolha:

- Algoritmo amplamente utilizado e com forte resistência a colisões.
- Opera de forma rápida, estável e criptograficamente segura.
- Adequado para geração de IDs persistentes e irreversíveis.

---

### Impacto na Análise

Após a criação do identificador anonimizado:

- O nome deixa de ser utilizado em qualquer cálculo, agrupamento ou visualização.
- Todas as análises passam a utilizar exclusivamente o campo **`id_servidor`**.
- Variáveis derivadas, como **sexo inferido**, **proventos**, **líquido**, **carga horária** e outras, permanecem disponíveis para análises sem risco de identificação.
- A estrutura resultante permite replicar o processo em outros conjuntos de dados com mínima adaptação.

Essa abordagem assegura um equilíbrio adequado entre **qualidade analítica**, **reprodutibilidade** e **responsabilidade ética** no uso dos dados públicos.

In [25]:
def gerar_id_servidor(nome):
    if pd.isna(nome):
        return None
    return hashlib.sha256(nome.encode("utf-8")).hexdigest()

df_prepared["id_servidor"] = df_prepared["nome_servidor_norm"].apply(gerar_id_servidor)

In [26]:
df_prepared[["nome_servidor_norm", "id_servidor"]].head()

,nome_servidor_norm,id_servidor
0,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
1,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
2,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
3,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
4,ADAO RODRIGUES DE JESUS,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...


## Tratando a coluna cargo

In [27]:
df_prepared["cargo"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: cargo
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


Nesta coluna foi identificado 23997 linhas sem valores nulos

### Quantidade de cargos distintos

In [28]:
df_prepared["cargo"].nunique()

127

In [29]:
sorted(df_prepared["cargo"].unique())

['AGENTE ADMINISTRATIVO',
 'AGENTE COMUNITARIO DE SAUDE.',
 'AGENTE DE COMBATE AS ENDEMIAS',
 'AGENTE DE DESENVOLVIMENTO INFANTIL',
 'AGENTE DE SERVIÇOS DE SAÚDE',
 'AGENTE DE VIGILANCIA SANITARIA',
 'AJUDANTE DE ENCANADOR',
 'AJUDANTE DE PEDREIRO',
 'AJUDANTE DE SERVICOS DIVERSOS',
 'ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.c',
 'ASSESSOR DE GABINETE.c',
 'ASSESSOR DE IMPLEMENTAÇÃO DE POLÍTICAS PÚBLICAS.c',
 'ASSESSOR DE PLANEJAMENTO.c',
 'ASSISTENTE SOCIAL',
 'ATENDENTE SOCIAL',
 'AUXILIAR DE CRECHE',
 'AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL',
 'AUXILIAR DE DENTISTA PSF',
 'AUXILIAR DE ENFERMAGEM',
 'AUXILIAR DE FARMACIA',
 'AUXILIAR DE MANUTENCAO',
 'AUXILIAR DE MEDICOS DENTISTA',
 'AUXILIAR DE SERVICOS EXTERNOS',
 'BIBLIOTECARIA',
 'BORRACHEIRO',
 'CARPINTEIRO',
 'CHEFE DE GABINETE.c',
 'CIRURGIAO DENTISTA PSF',
 'CONSELHEIRO TUTELAR.',
 'CONTADOR',
 'CONTROLADOR INTERNO',
 'COVEIRO',
 'DENTISTA',
 'DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.c',

- A listagem completa dos cargos foi inspecionada visualmente para avaliar consistência, padronização e possíveis variações na nomenclatura.
- Um cargo foi considerado **comissionado** quando seu nome terminava com o sufixo **“.c”**, conforme padrão identificado na base original.

## Criando categoria por cargo

- Normalizando a coluna cargo

In [30]:
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ASCII", "ignore").decode("ASCII")

    texto = texto.upper()

    texto = texto.strip()
    texto = re.sub(r"\s+", " ", texto)


    return texto

df_prepared["cargo_norm"] = df_prepared["cargo"].apply(normalizar_texto)

In [31]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,NaN,40.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,30.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,NaN,30.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO


- Mapeando os cargos

In [32]:
mapa_categoria_cargo = {
    # ADMINISTRATIVO / GESTAO
    "AGENTE ADMINISTRATIVO": "administrativo",
    "OFICIAL ADMINISTRATIVO": "administrativo",
    "CONTADOR": "administrativo",
    "CONTROLADOR INTERNO": "administrativo",
    "GESTOR DE PLANEJAMENTO": "administrativo",
    "FISCAL": "administrativo",

    # SAUDE
    "AGENTE COMUNITARIO DE SAUDE.": "saude",
    "AGENTE DE COMBATE AS ENDEMIAS": "saude",
    "AGENTE DE SERVICOS DE SAUDE": "saude",
    "AGENTE DE VIGILANCIA SANITARIA": "saude",
    "AUXILIAR DE DENTISTA PSF": "saude",
    "AUXILIAR DE ENFERMAGEM": "saude",
    "AUXILIAR DE FARMACIA": "saude",
    "AUXILIAR DE MEDICOS DENTISTA": "saude",
    "CIRURGIAO DENTISTA PSF": "saude",
    "DENTISTA": "saude",
    "ENFERMEIRO": "saude",
    "FARMACEUTICO": "saude",
    "FISIOTERAPEUTA": "saude",
    "FONOAUDIOLOGA": "saude",
    "MEDICO": "saude",
    "MEDICO ANESTESISTA": "saude",
    "MEDICO CARDIOLOGISTA": "saude",
    "MEDICO CIRURGIAO": "saude",
    "MEDICO CLINICO GERAL": "saude",
    "MEDICO DO TRABALHO": "saude",
    "MEDICO ENDOCRINOLOGISTA": "saude",
    "MEDICO GINECOLOGISTA/OBSTETRA": "saude",
    "MEDICO NEUROLOGISTA ADULTO": "saude",
    "MEDICO ORTOPEDISTA / TRAUMATOLOGISTA": "saude",
    "MEDICO PEDIATRA": "saude",
    "MEDICO PRONTO ATENDIMENTO": "saude",
    "MEDICO PSF": "saude",
    "MEDICO PSIQUIATRA ADULTO": "saude",
    "MEDICO UROLOGISTA": "saude",
    "NUTRICIONISTA": "saude",
    "TECNICO EM ENFERMAGEM": "saude",
    "TECNICO EM NUTRICAO": "saude",
    "VETERINARIO": "saude",

    # EDUCACAO
    "AGENTE DE DESENVOLVIMENTO INFANTIL": "educacao",
    "AUXILIAR DE CRECHE": "educacao",
    "INSPETOR DE ALUNOS": "educacao",
    "MERENDEIRA": "educacao",
    "MONITOR DE EDUCACAO FISICA": "educacao",
    "MESTRE DE MUSICA": "educacao",
    "PROFESSOR": "educacao",
    "PROFESSOR DE EDUCACAO FISICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - ARTE - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - CIENCIAS - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - EDUCACAO FISICA PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - GEOGRAFIA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INFORMATICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INGLES - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - MATEMATICA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - PORTUGUES - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL - PEI - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL -PEI": "educacao",
    "PROFESSOR DE ENSINO ESPECIAL": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA) EDUCACAO ESPECIAL - PD": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA-EDUC.ESPECIAL)": "educacao",
    "PROFESSOR EDUC.INFANTIL(CRECHE)-PEI-C": "educacao",
    "PROFESSOR EDUCACAO BASICA II - ARTE": "educacao",
    "PROFESSOR EDUCACAO BASICA II - EDUCACAO FISICA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - GEOGRAFIA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - LINGUA PORTUGUESA": "educacao",
    "PROFESSOR ENS FUND-CICLO I - II": "educacao",
    "PROFESSOR ENS FUND-CICLO III - IV": "educacao",
    "PROFESSOR SALA DE APOIO (PSA) EDUCACAO ESPECIAL": "educacao",

    # ASSISTENCIA SOCIAL
    "ASSISTENTE SOCIAL": "assistencia_social",
    "ATENDENTE SOCIAL": "assistencia_social",
    "ORIENTADOR SOCIAL": "assistencia_social",
    "CONSELHEIRO TUTELAR.": "assistencia_social",
    "AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL": "assistencia_social",
    "SUPERVISOR DE VISITAS": "assistencia_social",
    "PSICOLOGO": "assistencia_social",
    "PSICOLOGO INFANTIL": "assistencia_social",

    # OPERACIONAL / SERVICOS GERAIS
    "AJUDANTE DE ENCANADOR": "operacional",
    "AJUDANTE DE PEDREIRO": "operacional",
    "AJUDANTE DE SERVICOS DIVERSOS": "operacional",
    "AUXILIAR DE MANUTENCAO": "operacional",
    "AUXILIAR DE SERVICOS EXTERNOS": "operacional",
    "BORRACHEIRO": "operacional",
    "CARPINTEIRO": "operacional",
    "COVEIRO": "operacional",
    "ELETRICISTA": "operacional",
    "ENCANADOR": "operacional",
    "JARDINEIRO": "operacional",
    "LEITURISTA": "operacional",
    "LIXEIRO": "operacional",
    "MECANICO II": "operacional",
    "MOTORISTA": "operacional",
    "OPERADOR DA EBA": "operacional",
    "OPERADOR DE MAQUINA II": "operacional",
    "OPERADOR DE VACA MECANICA": "operacional",
    "PADEIRO": "operacional",
    "PEDREIRO": "operacional",
    "SERVENTE": "operacional",
    "VIGIA": "operacional",

    # TECNICO
    "TECNICO EM INFORMATICA": "tecnico",
    "TECNICO EM QUIMICA INDUSTRIAL (ETE)": "tecnico",
    "TECNICO SEGURANCA DO TRABALHO": "tecnico",

    # CULTURA / TURISMO
    "BIBLIOTECARIA": "cultura",
    "TURISMOLOGO": "cultura",

    # JURIDICO
    "PROCURADOR JURIDICO": "juridico",

    # POLITICO / ALTA GESTAO
    "PREFEITO": "politico",
    "VICE PREFEITO": "politico",

    # COMISSIONADOS / CARGOS DE CONFIANCA
    "ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.C": "comissionado",
    "ASSESSOR DE GABINETE.C": "comissionado",
    "ASSESSOR DE IMPLEMENTACAO DE POLITICAS PUBLICAS.C": "comissionado",
    "ASSESSOR DE PLANEJAMENTO.C": "comissionado",
    "CHEFE DE GABINETE.C": "comissionado",
    "GESTOR ADJUNTO DE ENSINO FUNDAMENTAL.C": "comissionado",
    "PROCURADOR GERAL DO MUNICIPIO.C": "comissionado",
    "DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ADMINISTRACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE AGRICULTURA E MEIO AMBIENTE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ASSISTENCIA SOCIAL.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO ECONOMICO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO URBANO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE EDUCACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE FINANCAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE OBRAS E ENGENHARIA.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SAUDE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SERVICOS MUNICIPAIS.C": "comissionado",
}


In [33]:
df_prepared["categoria_cargo"] = df_prepared["cargo_norm"].map(mapa_categoria_cargo)

In [34]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional


- Validando se algum cargo ficou sem catgoria

In [35]:
# Validando se algum cargo ficou sem categoria

df_prepared["categoria_cargo"].isna().sum()

np.int64(0)

In [36]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [37]:
(
    df_prepared["categoria_cargo"]
    .isna()
    .mean()
    * 100
)


np.float64(0.0)

In [38]:
df_prepared["cargo_categorizado"] = df_prepared["categoria_cargo"].notna()

In [39]:
df_prepared["cargo_categorizado"].value_counts()

cargo_categorizado
True    23997
Name: count, dtype: int64

In [40]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [41]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


- Conforme visto acima, todos os cargos foram mapeados

## Tratando coluna data_admissao

- Verificando nulos

In [42]:
df_prepared["data_admissao"].isna().sum()

np.int64(0)

In [43]:
df_prepared["data_admissao"].describe()

count          23997
unique           440
top       03/02/2025
freq            1066
Name: data_admissao, dtype: object

- Transformando o tipo da coluna para date

In [44]:
df_prepared["data_admissao"] = pd.to_datetime(
    df_prepared["data_admissao"],
    errors="coerce",
    dayfirst=True
)

In [45]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


In [46]:
type(df_prepared['data_admissao'].dropna().iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

## Criação da Flag de Identificação de Agentes Políticos

Durante a análise dos dados da folha de pagamento, observou-se que os cargos de **Prefeito** e **Vice-Prefeito** possuem características administrativas e remuneratórias distintas dos demais servidores públicos, por se tratarem de **agentes políticos**. Além disso, esses cargos podem apresentar datas de desligamento que não representam necessariamente a saída efetiva do mandato, mas sim procedimentos administrativos internos do sistema de folha.

Para tratar adequadamente essas particularidades, foi criada a variável auxiliar **`flag_agente_politico`**, com o objetivo de identificar registros associados a cargos eletivos sem removê-los da base original.

A flag assume os seguintes valores:

- `True` — quando o cargo corresponde a **Prefeito** ou **Vice-Prefeito**  
- `False` — para todos os demais servidores

### Benefícios da abordagem

- **Preserva** a integridade do dataset original, mantendo todos os registros.  
- Permite aplicar **tratamentos diferenciados** para agentes políticos quando necessário.  
- Facilita a **exclusão controlada** desses cargos em análises exploratórias e visualizações, evitando distorções estatísticas.  
- Ajuda a distinguir **vínculos administrativos** de alterações reais no exercício do cargo.

### Critério de Identificação

A detecção foi realizada por meio de correspondência textual na coluna `Cargo`, utilizando padrão que contempla o termo **“PREFEITO”**, garantindo a captura de variações como “Prefeito”, “VICE-PREFEITO” ou formatos semelhantes.

In [47]:
df_prepared["flag_agente_politico"] = df_prepared["cargo"].str.contains(
    r'\bPREFEITO\b',
    case=False,
    na=False
)

In [48]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


In [49]:
df_prefeito_vice = df_prepared[df_prepared['flag_agente_politico'] == True]

df_prefeito_vice[['nome', 'cargo', 'data_admissao', 'data_desligamento']].drop_duplicates()

,nome,cargo,data_admissao,data_desligamento
285,ARIANA APARECIDA PUPO,VICE PREFEITO,2025-01-01,NaN
1256,MARCELO SIMAO,PREFEITO,2025-01-01,NaN
9685,MARCELO SIMAO,PREFEITO,2021-01-01,01/01/2025
9706,MARCIO RENATO CANDIDO DOS REIS,VICE PREFEITO,2021-01-01,01/01/2025


## Tratando coluna tipo_regime

In [50]:
df_prepared["tipo_regime"].describe

<bound method NDFrame.describe of 0        REGIME PROPRIO
1        REGIME PROPRIO
2        REGIME PROPRIO
3        REGIME PROPRIO
4        REGIME PROPRIO
              ...      
23992    REGIME PROPRIO
23993    REGIME PROPRIO
23994    REGIME PROPRIO
23995    REGIME PROPRIO
23996    REGIME PROPRIO
Name: tipo_regime, Length: 23997, dtype: object>

In [51]:
sorted(df_prepared["tipo_regime"].unique())

['REGIME GERAL  ', 'REGIME PROPRIO']

In [52]:
df_prepared["tipo_regime"] = df_prepared["tipo_regime"].apply(normalizar_texto)

In [53]:
df_prepared["tipo_regime"].unique()

array(['REGIME PROPRIO', 'REGIME GERAL'], dtype=object)

## Considerações sobre o Regime Previdenciário

No contexto dos servidores públicos municipais e estaduais, a coluna que identifica o **regime previdenciário** apresenta informações relevantes para interpretação administrativa, mas não requer transformação direta nesta etapa. A seguir, detalha-se o significado dos dois regimes presentes:

### Regime Geral de Previdência Social (RGPS)
- Geralmente associado a servidores **efetivos**, ingressantes via concurso público.
- Os servidores contribuem para o **INSS**.
- Representa vínculos com **estabilidade** e direitos estruturados pelas regras do serviço público.

### Regime Próprio de Previdência Social (RPPS)
- Pode incluir tanto servidores **efetivos** quanto **comissionados**, variando conforme legislação municipal/estadual.
- Para cargos **comissionados**, o vínculo é de livre nomeação e exoneração, sem estabilidade.
- Os servidores contribuem para o **regime próprio** do ente federativo (municipal ou estadual).

---

### Decisão de Preparação dos Dados

Apesar da relevância contextual, a coluna do regime previdenciário apresenta valores já padronizados e semanticamente corretos. Portanto:

- **Será mantido o formato original**,  
- **Não serão aplicadas transformações**,  
- E seu uso será considerado apenas em análises onde o regime possa desempenhar papel interpretativo.

Essa decisão evita alterações desnecessárias e preserva a integridade semântica dos dados.

### Tratamento da Coluna `desconto`

A coluna **`desconto`** representa os valores descontados do pagamento do servidor. Na base original, ela apresenta comportamento distinto conforme o tipo de pagamento, o que exige tratamento para padronização e consistência analítica.

#### Observações iniciais
- Para registros referentes a **vale-alimentação**, não há desconto associado; por isso, a coluna aparece como `NaN`.
- Para registros referentes à **folha mensal**, o desconto é informado como texto no **formato brasileiro**, por exemplo: `1.500,00`.

#### Objetivos do tratamento
1. Substituir `NaN` por **zero** nos pagamentos do tipo **vale_alimentacao**, preservando o significado “sem desconto”.
2. Converter valores monetários textuais (pt-BR) para o tipo **`float`**, permitindo cálculos financeiros consistentes.
3. Garantir que a coluna fique **sem valores nulos** após o processamento.

#### Passo a passo

1. **Identificar o tipo de pagamento**
   - A coluna **`tipo_pagamento`** indica se o registro corresponde a `folha_mensal` ou `vale_alimentacao`.

2. **Criar função de conversão (pt-BR → float)**
   - Implementar uma função para transformar strings monetárias no padrão brasileiro em valores numéricos (`float`), lidando corretamente com separadores de milhar e decimal.

In [54]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0

    return float(str(valor).replace(".", "").replace(",", "."))

df_prepared["descontos"] = np.where(
    df_prepared["tipo_pagamento"] == "vale_alimentacao", 0.0,
    df_prepared["descontos"].apply(str_para_float)
)

In [55]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,"3.603,75",NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,"5.432,23",NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


In [56]:
df_prepared["descontos"].describe

<bound method NDFrame.describe of 0           0.00
1        1584.02
2           0.00
3        1330.84
4           0.00
          ...   
23992     535.56
23993       0.00
23994     323.08
23995    1947.27
23996       0.00
Name: descontos, Length: 23997, dtype: float64>

In [57]:
df_prepared["descontos"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: descontos
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


### Tratamento da Coluna `liquido`

A coluna **`liquido`** representa o valor efetivamente recebido pelo servidor em cada pagamento. Na base original, os valores estão representados como texto no **formato monetário brasileiro**, o que impede cálculos financeiros diretos.

#### Observações iniciais
- Os valores utilizam o padrão brasileiro, por exemplo: `1.500,00`.
- Para análises financeiras, é necessário convertê-los para o tipo **`float`** no formato internacional (`1500.00`).
- Diferentemente da coluna `desconto`, **todos os registros de `liquido` devem ser convertidos**, independentemente do tipo de pagamento.

#### Objetivos do tratamento
1. Converter todos os valores da coluna para o tipo **`float`**, aplicando formatação numérica padronizada.
2. Garantir que não existam valores **`NaN`** após o processo de conversão.
3. Padronizar a coluna para permitir **análises financeiras**, cálculos agregados e comparações entre registros.

In [58]:
df_prepared["liquido"].describe

<bound method NDFrame.describe of 0        1.000,00
1        3.603,75
2        1.000,00
3        5.432,23
4        1.000,00
           ...   
23992    2.920,63
23993    1.000,00
23994    2.442,78
23995    3.576,00
23996    1.000,00
Name: liquido, Length: 23997, dtype: object>

In [59]:
df_prepared["liquido"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: liquido
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [60]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0
    # Substitui ponto como separador de milhar e vírgula como decimal
    return float(str(valor).replace('.', '').replace(',', '.'))

df_prepared["liquido"] = df_prepared["liquido"].apply(str_para_float)

In [61]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


## Tratamento da Coluna `data_desligamento`

A coluna **`data_desligamento`** indica a data em que o servidor foi desligado de sua função. Durante a análise inicial, observou-se que a maior parte dos registros apresenta valor nulo (`NaN`), o que é esperado no contexto da administração pública, já que a maioria dos servidores permanece ativa por longos períodos. Desligamentos, quando ocorrem, são mais comuns entre cargos comissionados ou situações específicas.

Para lidar corretamente com essa característica e garantir consistência semântica, foi adotado o seguinte processo de tratamento:

### Processo de Tratamento

1. **Criação de coluna auxiliar (`data_desligamento_formatada`)**  
   - A coluna original é preservada sem alterações, mantendo o dado bruto para referência futura.

2. **Representação explícita de servidores ativos**  
   - Valores nulos (`NaN`) foram substituídos pelo símbolo `"-"`, indicando formalmente que o servidor permanece **ativo**.  
   - Essa decisão evita interpretações equivocadas em análises subsequentes.

3. **Conversão de valores válidos para data**  
   - Registros contendo datas foram convertidos para o tipo `date` com o padrão **ISO (`YYYY-MM-DD`)**, garantindo consistência e facilitando cálculos temporais.

4. **Apoio a análises futuras**  
   - Essa padronização permite análises como:
     - identificação de servidores desligados,
     - cálculo de tempo de serviço,
     - análise comparativa entre vínculos ativos e inativos,
     - geração de métricas temporais confiáveis.

### Exemplo de Resultado

| data_desligamento | data_desligamento_formatada |
|-------------------|-----------------------------|
| NaN               | -                           |
| 15/08/2020        | 2020-08-15                  |

Essa abordagem mantém a **integridade do dado original**, melhora a **clareza analítica** e prepara o campo para uso adequado nas etapas de EDA e modelagem.

In [62]:
def formatar_desligamento(data):
    if pd.isna(data):
        return pd.NaT
    else:
        return pd.to_datetime(data, dayfirst=True).date()

df_prepared["data_desligamento_formatada"] = df_prepared["data_desligamento"].apply(formatar_desligamento)

In [63]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT


In [64]:
df_prepared["data_desligamento_formatada"].nunique()

73

In [65]:
df_prepared["data_desligamento_formatada"].value_counts()

data_desligamento_formatada
2025-12-12    54
2025-06-02     7
2025-01-01     4
2025-07-01     4
2025-05-05     4
              ..
2025-10-02     1
2025-09-09     1
2025-09-25     1
2025-09-26     1
2025-09-01     1
Name: count, Length: 73, dtype: int64

In [66]:
df_prepared["status_servidor"] = df_prepared["data_desligamento"].apply(
    lambda x: "ATIVO" if pd.isna(x) else "DESLIGADO"
)

In [67]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada,status_servidor
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT,ATIVO


- **Servidores desligados**

A partir da coluna `data_desligamento_formatada`, foi identificado que **149 servidores** possuem registro de desligamento no período analisado. 

Para garantir consistência, essa contagem considera **apenas servidores únicos**, evitando duplicidades decorrentes de múltiplos registros mensais de pagamento para um mesmo indivíduo.

### Tratamento da Coluna `proventos`

A coluna **`proventos`** representa o valor bruto recebido pelo servidor. Na base original, os valores estavam armazenados como texto (`object`) utilizando a **notação monetária brasileira** (ex.: `1.500,00`), o que inviabiliza operações numéricas diretas.

### Procedimento de tratamento

1. **Remoção do separador de milhar**  
   - O ponto (`.`) utilizado como separador de milhares foi removido para padronização.

2. **Substituição da vírgula decimal**  
   - A vírgula (``,`), padrão brasileiro de separação decimal, foi substituída por ponto (`.`), permitindo compatibilidade com o formato numérico internacional.

3. **Conversão para tipo `float`**  
   - Após a normalização textual, os valores foram convertidos para o tipo `float`, tornando a coluna adequada para operações quantitativas.

### Resultado

Com esse tratamento, a coluna `proventos` passa a estar devidamente estruturada para:

- Cálculos de média, soma, mínimo e máximo  
- Análises comparativas entre categorias e cargos  
- Avaliações temporais do gasto total com pessoal  
- Modelagens estatísticas e financeiras  

In [68]:
df_prepared["proventos"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: proventos
Non-Null Count  Dtype 
--------------  ----- 
23919 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [69]:
df_prepared["proventos"] = (
    df_prepared["proventos"]
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [70]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada,status_servidor
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,5187.77,1.0,...,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,6763.07,1.0,...,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT,ATIVO


## Coluna `contrato`

A coluna **`contrato`** apresenta apenas dois valores distintos: **1.0** e **2.0**.  
Em bases de gestão de pessoal do setor público, é comum que esses números representem **códigos internos** utilizados para diferenciar tipos de vínculo funcional. A interpretação observada é:

- **1** → Servidor efetivo ou concursado  
  (regime estatutário ou vínculo CLT de longo prazo)

- **2** → Contrato temporário ou vínculo especial  
  (contratação emergencial, temporária ou cargo comissionado)

Embora a coluna seja relevante do ponto de vista administrativo, não exige transformação nesta etapa, mas será utilizada posteriormente para análises comparativas entre diferentes tipos de vínculo.

In [71]:
df_prepared["contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: contrato
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


In [72]:
df_prepared["contrato"] = df_prepared["contrato"].astype("Int64")

In [73]:
df_prepared["contrato"].value_counts()

contrato
1    22714
2     1283
Name: count, dtype: Int64

**Decisão adotada:**  
A coluna `contrato` será mantida no dataset por questões de integridade, mas **não será utilizada nas análises**, pois representa apenas um código interno com baixo poder explicativo para os objetivos deste projeto.

## Coluna `atividade`

A coluna **`atividade`** apresenta um número reduzido de registros preenchidos e, quando presente, contém apenas valores numéricos. Isso indica que se trata de **códigos internos** utilizados pelo sistema da prefeitura para classificar atividades ou funções específicas.

### Observações
- Não há documentação disponível que permita interpretar o significado de cada código.
- A baixa taxa de preenchimento reduz seu potencial analítico.
- A coluna não apresenta relevância para os objetivos deste projeto.

In [74]:
df_prepared["atividade"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: atividade
Non-Null Count  Dtype  
--------------  -----  
24 non-null     float64
dtypes: float64(1)
memory usage: 187.6 KB


In [75]:
df_prepared["atividade"] = df_prepared["atividade"].astype("Int64")

In [76]:
df_prepared["atividade"].value_counts()

atividade
10    24
Name: count, dtype: Int64

**Decisão adotada:**  
A coluna `atividade` não será utilizada nas análises, devido à baixa taxa de preenchimento e à ausência de documentação sobre o significado dos códigos.  
Caso futuramente seja disponibilizado um dicionário oficial desses códigos, a coluna poderá ser mapeada e incorporada às análises de forma estruturada.

## Coluna `nome_atividade`

A coluna **`nome_atividade`** apresenta um número muito reduzido de registros preenchidos.  
Além disso, nos casos em que possui informação, seu conteúdo é redundante, pois as descrições já estão contempladas na coluna **`cargo`**. Assim, ela não adiciona valor informativo ao dataset.

In [77]:
df_prepared["nome_atividade"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: nome_atividade
Non-Null Count  Dtype 
--------------  ----- 
24 non-null     object
dtypes: object(1)
memory usage: 187.6+ KB


**Decisão adotada:**  
Assim como a coluna `atividade`, a coluna `nome_atividade` **não passará por tratamento nesta etapa**, devido à sua redundância e baixa taxa de preenchimento.  
Caso futuramente seja identificado algum uso específico para essas informações, a coluna poderá ser revisitada e integrada de forma adequada às análises.

## Coluna `tipo_contrato`

A coluna **`tipo_contrato`** descreve a natureza do vínculo do servidor com a prefeitura. Os valores identificados na base são:

- **Efetivo**
- **Contrato Temporário**
- **Função de Confiança**
- **Efetivo em Comissão**

Essa variável possui potencial analítico relevante, especialmente para a diferenciação entre vínculos permanentes, temporários e cargos de confiança.

In [78]:
df_prepared["tipo_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: tipo_contrato
Non-Null Count  Dtype 
--------------  ----- 
23984 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [79]:
df_prepared["tipo_contrato"].unique()

array(['Efetivo', 'Função de Confiança', 'Contrato Temporario',
       'Efetivo em Comissão', nan, 'Contrato/Temporário'], dtype=object)

**Observações:**

- A coluna apresenta poucos valores nulos.
- A maioria dos registros está consistente e com nomenclatura padronizada.
- Os valores são suficientemente descritivos para uso direto em análises.

**Decisão adotada:**  
Nenhum tratamento será aplicado à coluna neste momento, pois os dados já se encontram adequados para análise.  
Caso futuramente surja a necessidade de agrupar, recodificar ou padronizar categorias específicas, a coluna poderá ser revisitada.

## Coluna `data_prevista_termino_contrato`

In [80]:
df_prepared["data_prevista_termino_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: data_prevista_termino_contrato
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 187.6 KB


In [81]:
df_prepared["data_prevista_termino_contrato"] = pd.to_datetime(
    df_prepared["data_prevista_termino_contrato"],
    errors="coerce",
    dayfirst=True
)

In [82]:
df_prepared["data_prevista_termino_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: data_prevista_termino_contrato
Non-Null Count  Dtype         
--------------  -----         
0 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 187.6 KB


**Observações:**

- Todos os valores da coluna estão nulos.
- Não há qualquer informação preenchida em nenhuma linha do dataset.
- A coluna não possui utilidade analítica no estado atual.

**Decisão adotada:**  
A coluna será mantida no dataset apenas para preservar a integridade estrutural, mas **não será utilizada nas análises**, pois não contém dados relevantes.  
Caso futuramente seja disponibilizada documentação oficial ou apareçam registros válidos, a coluna poderá ser revisitada ou descartada sem impacto no projeto.

## Coluna `carga_horaria_semanal`

### Observações

- A coluna representa a **carga horária semanal** atribuída a cada servidor.
- Embora os valores tenham sido importados como `float`, todos representam **horas inteiras**, sem casas decimais.
- A variável possui potencial analítico relevante para comparações entre categorias, cargos e tipos de vínculo.

### Tratamento realizado

- A coluna foi convertida de `float` para **`int`**, refletindo corretamente a natureza do dado e facilitando análises estatísticas e agregações futuras.

In [83]:
df_prepared["carga_horaria_semanal"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: carga_horaria_semanal
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


In [84]:
df_prepared["carga_horaria_semanal"] = df_prepared["carga_horaria_semanal"].astype("Int64")

In [85]:
df_prepared["carga_horaria_semanal"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: carga_horaria_semanal
Non-Null Count  Dtype
--------------  -----
23997 non-null  Int64
dtypes: Int64(1)
memory usage: 211.0 KB


## Ajustes finais

- Renomeando colunas

In [86]:
df_prepared = df_prepared.rename(columns={
    "mes_referencia": "mes",
    "sexo_inferido": "sexo",
    "data_desligamento_formatada": "data_desligamento_",
    "tipo_pagamento_normalizado": "tipo_pagamento_"
})

- Aplicando filtro para pegar Prefeito e Vice Prefeito atuais

In [87]:
df_filtered = df_prepared[
    (~df_prepared['flag_agente_politico']) |
    (
        (df_prepared['flag_agente_politico']) &
        (df_prepared['data_admissao'] >= '2025-01-01')
    )
].copy()

- Selecionando colunas a serem exportadas

In [88]:
colunas_final = [
    "id_servidor",
    "sexo",
    "cargo",
    "categoria_cargo",
    "tipo_pagamento_",
    "proventos",
    "descontos",
    "liquido",
    "carga_horaria_semanal",
    "data_admissao",
    "data_desligamento_",
    "status_servidor",
    "mes"
]

- Validação rápida

In [89]:
print("df_prepared:", df_prepared.shape)
print("df_filtered:", df_filtered.shape)

df_prepared: (23997, 28)
df_filtered: (23995, 28)


- Garantir que políticos antigos não ficaram

In [90]:
df_filtered[
    (df_filtered["cargo"].str.contains("PREFEITO", case=False, na=False)) &
    (df_filtered["data_admissao"] < "2025-01-01")
]

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_,nome_servidor_norm,sexo,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_,status_servidor


- Criação do df_final

In [91]:
df_final = df_filtered[colunas_final].copy()

- Tratamento da variável `mes`

In [92]:
def normalizar_mes(mes):
    if pd.isna(mes):
        return None

    mes = mes.strip().lower()
    mes = unicodedata.normalize("NFKD", mes).encode("ascii", "ignore").decode("utf-8")

    return mes[:3]

In [93]:
df_final["mes"] = df_final["mes"].apply(normalizar_mes)

- Ordenação categórica dos meses

In [94]:
ordem_meses = ["jan", "fev", "mar", "abr", "mai", "jun",
               "jul", "ago", "set", "out", "nov", "dez"]

df_final["mes"] = pd.Categorical(
    df_final["mes"],
    categories=ordem_meses,
    ordered=True
)

df_final = df_final.sort_values("mes").reset_index(drop=True)

- Renomeando coluna no df_final

In [95]:
df_final = df_final.rename(columns={
    "sexo": "genero",
    "data_desligamento_": "data_desligamento",
    "tipo_pagamento_": "tipo_pagamento"
})

- Convertendo coluna `data_desligamento` para datetime

In [96]:
df_final["data_desligamento"] = pd.to_datetime(
    df_final["data_desligamento"],
    errors="coerce",
    dayfirst=True
)

- Exportação para EDA

In [97]:
output_path = "../data/processed/servidores2025.csv"

df_final.to_csv(
    output_path,
    index=False,
    encoding="utf-8"
)

- Exportação parquet

In [99]:
output_path_parquet = "../data/processed/folha-pagamento-2025"

df_final.to_parquet(
    output_path_parquet,
    index=False
)

## Considerações Finais

Este notebook conclui a etapa de **Data Preparation** da metodologia **CRISP-DM**, cujo objetivo é transformar os dados brutos em uma base estruturada, consistente e pronta para análise. Todo o trabalho realizado aqui garante que as próximas etapas possam ser conduzidas com rigor técnico, reprodutibilidade e clareza metodológica.

Ao longo do processo, foram executados os seguintes procedimentos principais:

- **Padronização de valores monetários** (`proventos`, `descontos`, `liquido`), convertendo do formato brasileiro para tipos numéricos adequados.
- **Conversão e padronização de datas**, preservando informações originais e criando campos normalizados (`date`) para cálculos temporais.
- **Inferência e consolidação da variável `sexo_inferido`**, com documentação completa do processo.
- **Anonimização dos servidores**, por meio da criação de identificadores únicos (`id_servidor`) usando hash criptográfico.
- **Normalização e categorização de cargos**, incluindo identificação de comissionados e agentes políticos.
- **Avaliação criteriosa de colunas auxiliares**, com decisões documentadas sobre manter, descartar ou não utilizar determinadas variáveis.
- **Ajustes estruturais finais**, como seleção de colunas relevantes, ordenação do dataset e criação do DataFrame final (`df_final`).

Como resultado, foi gerado um **arquivo CSV estruturado**, salvo em `data/processed`, que representa a **base oficial** para uso nas próximas fases do projeto.

---

### O dataset final foi construído com foco em:

- **Reprodutibilidade** — processos documentados e replicáveis.  
- **Transparência metodológica** — decisões e transformações justificadas.  
- **Facilidade de análise** — dados prontos para cálculos, agrupamentos e visualizações.  
- **Responsabilidade no uso dos dados** — anonimização e boas práticas éticas.  

---

### Próximos Passos

Com a etapa de preparação concluída, os dados estão prontos para:

- Análises exploratórias (EDA)  
- Geração de indicadores e métricas  
- Construção de visualizações  
- Comunicação pública dos resultados  
- Criação de aplicações interativas (ex.: Streamlit)

A etapa seguinte será conduzida em um notebook dedicado exclusivamente à **Exploratory Data Analysis (EDA)**, mantendo a separação clara entre **preparação** e **interpretação** dos dados, conforme as melhores práticas do CRISP-DM.